In [3]:
s = 1e-5

import pandas as pd
from sklearn import metrics
import tensorflow as tf
################################ MSE ################################
def MSE(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true-y_pred))

################################ BCE ################################
def BCE(y_true, y_pred):
    return -tf.reduce_mean(y_true*tf.math.log(y_pred+s)+(1-y_true)*tf.math.log(1-y_pred+s))

################################ WBCE ################################
def WBCE(y_true, y_pred):
    N = batch    # batch_size
    y1 = tf.reduce_sum(y_true)
    y0 = N-y1
    w1 = y0/N #N/y1
    w0 = y1/N #N/y0
    return -tf.reduce_mean(w1*y_true*tf.math.log(y_pred+s)+w0*(1-y_true)*tf.math.log(1-y_pred+s))

################################ TN/FP/FN/TP ################################
def confusion_matrix(y_true, y_pred):
    N = batch    # batch_size
    y1 = tf.reduce_sum(y_true)
    y0 = N-y1
    TN = N-tf.reduce_sum(y_true)-tf.reduce_sum(y_pred)+tf.reduce_sum(y_true*y_pred)
    FP = tf.reduce_sum(y_pred)-tf.reduce_sum(y_true*y_pred)
    FN = tf.reduce_sum(y_true)-tf.reduce_sum(y_true*y_pred)
    TP = tf.reduce_sum(y_true*y_pred)
    return N, y1, y0, TN, FP, FN, TP

################################ make_lists ################################
def make_lists():
    list_acc = []
    list_f1 = []
    list_gmean = []
    list_bacc = []
    list_pre = []
    list_rec = []
    list_spe = []
    return list_acc, list_f1, list_gmean, list_bacc, list_pre, list_rec, list_spe
    
############################### Results ###############################
def get_results(y_true, y_pred):
    TN = metrics.confusion_matrix(y_true, y_pred)[0,0]
    FP = metrics.confusion_matrix(y_true, y_pred)[0,1]
    FN = metrics.confusion_matrix(y_true, y_pred)[1,0]
    TP = metrics.confusion_matrix(y_true, y_pred)[1,1]
    acc = np.round((TP+TN)/(TP+TN+FP+FN),4)
    if TP+FP == 0:
        pre = 0
    else:
        pre = np.round(TP/(TP+FP),4)
    rec = np.round(TP/(TP+FN),4)
    spe = np.round(TN/(TN+FP),4)
    f1 = np.round(TP/(TP + 0.5*(FP+FN)),4)
    f05 = np.round(TP/(TP + 0.8*FP + 0.2*FN),4)
    f2 = np.round(TP/(TP + 0.2*FP + 0.8*FN),4)
    gmean = np.round(((TP/(TP+FN)) * (TN/(TN+FP)))**0.5,4)
    bacc = np.round(0.5*(TP/(TP+FN) + TN/(TN+FP)),4)
    
    list_acc.append(acc)
    list_f1.append(f1)
    list_gmean.append(gmean)
    list_bacc.append(bacc)
    list_pre.append(pre)
    list_rec.append(rec)
    list_spe.append(spe)

################################ SPL ################################
def splitter(y_pred):
    return (0.5)**2-(y_pred-0.5)**2

# =================================== Fbeta =================================== #
################################ Pure_Fbeta ################################
def Pure_Fbeta(y_true, y_pred):
    b = 1 
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return 1-F_beta

################################ Any_Fbeta ################################
def Any_Fbeta(y_true, y_pred):
    b = 1 
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return 1-F_beta

################################ WBCEFL ################################
def WBCEFL(y_true, y_pred):
    b = 1
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return (1-r)*WBCEloss+(r)*(1-F_beta)

################################ SPLFL ################################
def SPLFL(y_true, y_pred):
    b = 1
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return (1-w)*SPL+(w)*(1-F_beta)

# =================================== Gmean =================================== #
################################ Pure_Gmean ################################
def Pure_Gmean(y_true, y_pred):
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return 1-sur_gmean

################################ Any_Gmean ################################
def Any_Gmean(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return 1-sur_gmean

################################ WBCEGL ################################
def WBCEGL(y_true, y_pred):
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return (1-r)*WBCEloss+(r)*(1-sur_gmean)

################################ SPLGL ################################
def SPLGL(y_true, y_pred):
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return (1-w)*SPL+(w)*(1-sur_gmean)

# =================================== BAccu =================================== #
################################ Pure_BAccu ################################
def Pure_BAccu(y_true, y_pred):
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return 1-baccu

################################ Any_BAccu ################################
def Any_BAccu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return 1-baccu

################################ WBCEBL ################################
def WBCEBL(y_true, y_pred):
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return (1-r)*WBCEloss+(r)*(1-baccu)

################################ SPLBL ################################
def SPLBL(y_true, y_pred):
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return (1-w)*SPL+(w)*(1-baccu)

In [ ]:
res = pd.DataFrame({'MSE':[0, 0, 0, 0, 0, 0, 0]}, index = ['Acc','F1','G_Mean','B_Acc','Pre','Rec','Spe'])
res

# Step1: Select One Dataset

## 1)SPAM

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
#  https://www.kaggle.com/uciml/sms-spam-collection-dataset
data = pd.read_csv('spam.csv', encoding='latin1')
print('sample number:',len(data))

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[:5]

In [ ]:
data.drop_duplicates(subset=['v2'], inplace=True)

In [ ]:
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:350]
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
data.drop(data[:175][data['v1'][:175] == 1].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
X = data['v2']
y = data['v1']
y = y.astype(float)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 128
epochs = 20

## 2)Reuters News

In [ ]:
# https://www.kaggle.com/datasets/thedevastator/uncovering-financial-insights-with-the-reuters-2?select=ModApte_test.csv
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
data = pd.read_csv("ModHayes_train.csv")
data

In [ ]:
data = data[["text","topics"]]

In [ ]:
data['topics'].value_counts()[:10]

In [ ]:
isblank = data['topics'] == "[]"
isacq = data['topics'] == "['acq']"
data = data[isblank | isacq]
data = data.dropna()
data

In [ ]:
data['topics'] = data['topics'].replace(["[]","['acq']"],[0,1])
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:350]
data

In [ ]:
data['topics'].value_counts()

In [ ]:
data.drop(data[:200][data['topics'][:200] == 1].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
data['topics'].value_counts()

In [ ]:
X = data['text']
y = data['topics']
y = y.astype(float)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 64
epochs = 20

## 3)IMDB

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
data = pd.read_csv("IMDB Dataset.csv")
data

In [ ]:
data['sentiment'] = data['sentiment'].replace(['negative','positive'],[0,1])
data

In [ ]:
X_train = data['review'][:400]
y_train = data['sentiment'][:400]

In [ ]:
# Making lists of index
idx_0 = []
idx_1 = []
for i in range(len(y_train)):
    if list(y_train)[i] == 0:
        idx_0.append(i)
    if list(y_train)[i] == 1:
        idx_1.append(i)
print(len(idx_0), len(idx_1))

In [ ]:
idx_1_new = idx_1[:40]

idx = idx_0 + idx_1_new
idx.sort()
print(len(idx), idx)

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_train[i])
    y.append(y_train[i])
X = np.array(X)
y = np.array(y)
y = y.astype(float)
print(X.shape, y.shape)

In [ ]:
print(pd.Series(y).value_counts())

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 128
epochs = 20

# Step2: Do BERT

In [ ]:
from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

tfhub_handle_encoder ='https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
mse_acc = []
mse_f1 = []
mse_gmean = []
mse_bacc = []
mse_pre = []
mse_rec = []
mse_spe = []

for i in range(5):
    print('#'*50,'{0}th repeat'.format(i+1),'#'*50)
    list_acc, list_f1, list_gmean, list_bacc, list_pre, list_rec, list_spe = make_lists()

    n_iter=0
    ###################### MLP (sigmoid // MSE) ##############################
    for train_index, test_index in skf.split(X, y):
        n_iter += 1
        X_train = X[train_index]
        y_train= y[train_index]
        X_test = X[test_index]
        y_test= y[test_index]
#         print('#'*50,'{0}th CV'.format(n_iter),'#'*50)    
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
        y_train = tf.convert_to_tensor(y_train, dtype=tf.float32, dtype_hint=None, name=None)
        y_test = tf.convert_to_tensor(y_test, dtype=tf.float32, dtype_hint=None, name=None)

        model = build_classifier_model()
        early_stopping = EarlyStopping(monitor='loss', patience=0, min_delta=1e-4)
    #     check_point = ModelCheckpoint('best_model.h5', monitor="val_loss", save_best_only=True, save_weights_only=True)
        opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        model.compile(optimizer=opt, loss=MSE, metrics=['Accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), verbose=0, epochs=epochs, batch_size = batch, ) 
                            #callbacks=[early_stopping]) # , check_point])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = []
    #     model.load_weights('best_model.h5')  # Best Model by Check_Point
        result = model.predict(X_test)
        for i in range(X_test.shape[0]):
            if result[i] <= 0.5:
                predicted.append(0)
            else:
                predicted.append(1)
        get_results(y_test, predicted)
    print("Acc:{}\nF1:{}\nGM:{}\nBA:{}\nPRE:{}\nREC:{}\nSPE:{}\n".format(np.mean(list_acc),np.mean(list_f1),np.mean(list_gmean),
                                                                         np.mean(list_bacc),np.mean(list_pre),np.mean(list_rec),
                                                                         np.mean(list_spe)))     
    mse_acc.append(np.mean(list_acc))
    mse_f1.append(np.mean(list_f1))
    mse_gmean.append(np.mean(list_gmean))
    mse_bacc.append(np.mean(list_bacc))
    mse_pre.append(np.mean(list_pre))
    mse_rec.append(np.mean(list_rec))
    mse_spe.append(np.mean(list_spe))
               
res['MSE'] = [np.mean(mse_acc), np.mean(mse_f1), np.mean(mse_gmean), np.mean(mse_bacc), 
              np.mean(mse_pre), np.mean(mse_rec), np.mean(mse_spe)]
res 

In [ ]:
print("AC:", np.round(np.mean(mse_acc),4),'±',np.round(np.std(mse_acc),4))
print("F1:", np.round(np.mean(mse_f1),4),'±',np.round(np.std(mse_f1),4))
print("GM:", np.round(np.mean(mse_gmean),4),'±',np.round(np.std(mse_gmean),4))
print("BA:", np.round(np.mean(mse_bacc),4),'±',np.round(np.std(mse_bacc),4))
print("PRE:", np.round(np.mean(mse_pre),4),'±',np.round(np.std(mse_pre),4))
print("REC:", np.round(np.mean(mse_rec),4),'±',np.round(np.std(mse_rec),4))
print("SPE:", np.round(np.mean(mse_spe),4),'±',np.round(np.std(mse_spe),4))

In [ ]:
# For each loss, do above
# LOSS = [MSE, BCE, WBCE, Pure_Fbeta, Any_Fbeta, WBCEFL, SPLFL, 
#         Pure_Gmean, Any_Gmean, WBCEGL, SPLGL, 
#         Pure_BAccu, Any_BAccu, WBCEBL, SPLBL ]

In [ ]:
# Final Results
res